In [2]:
import polars as pl
import numpy as np
import pandas as pd
from module import network, util, dataset

In [5]:
# %%
import gc
import os
import pickle
import random
import joblib
import pandas as pd
# import polars as pd
from tqdm import tqdm
from sklearn.metrics import average_precision_score as APS
import polars as pl
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold
import numpy as np
import torch.nn.functional as F
import math
import time
import sys
import datetime
import pytz
import matplotlib.pyplot as plt


from module import network, dataset, util
from importlib import reload

# %%
class Config:
    PREPROCESS = False
    KAGGLE_NOTEBOOK = False
    DEBUG = False
    MODEL = 'lstm'
    SEED = 42
    EPOCHS = 9*2
    BATCH_SIZE = 4096
    LR = 1e-4
    WD = 1e-6
    PATIENCE = 3
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    EARLY_STOPPING = False
    NUM_CV = 1
    VAL_INDEX = [0]
    NOTEBOOK = False
    LOAD_MODEL = True
    # models配下
    MODEL_PATH = "lstm/lstm_fold0_72.pt"
    
    
if Config.DEBUG:
    n_rows = 10**4
else:
    n_rows = None



# %%
if Config.KAGGLE_NOTEBOOK:
    RAW_DIR = "/kaggle/input/leash-BELKA/"
    PROCESSED_DIR = "/kaggle/input/belka-enc-dataset"
    OUTPUT_DIR = ""
    MODEL_DIR = ""
else:
    RAW_DIR = "../data/raw/"
    PROCESSED_DIR = "../data/processed/"
    OUTPUT_DIR = "../data/result/"
    MODEL_DIR = "../models/"
    LOG_DIR = "../data/logs"
    LOSS_DIR = "../data/losses"

TRAIN_DATA_NAME = "train_enc.parquet"


In [13]:

def predict_in_batches(model, data, batch_size):
    model.eval()  # Set model to evaluation mode
    preds = []
    for i in range(0, data.size(0), batch_size):
        batch = data[i:i+batch_size].to(Config.DEVICE)
        with torch.no_grad():
            batch_preds = torch.sigmoid(model(batch))  # apply sigmoid
        preds.append(batch_preds.detach().cpu())
    return torch.cat(preds, dim=0)

In [14]:
model_path = os.path.join(MODEL_DIR, Config.MODEL_PATH)

In [15]:
model = network.LSTMModel().to(Config.DEVICE)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [18]:
# テストデータの読み込み
test = pd.read_parquet(os.path.join(PROCESSED_DIR, "test_enc.parquet"))
test_tensor = torch.tensor(test.values, dtype=torch.float32).to(Config.DEVICE)
preds = predict_in_batches(model, test_tensor, Config.BATCH_SIZE)

In [21]:
preds = preds.numpy()

In [22]:

tst = pl.read_parquet(os.path.join(RAW_DIR, "test.parquet"), n_rows=None).to_pandas()

# 'binds'列を追加して初期化
tst['binds'] = 0

# ブールマスクの作成
mask_BRD4 = (tst['protein_name'] == 'BRD4').values
mask_HSA = (tst['protein_name'] == 'HSA').values
mask_sEH = (tst['protein_name'] == 'sEH').values

# 各マスクに対応する予測値を代入
tst.loc[mask_BRD4, 'binds'] = preds[mask_BRD4][:, 0]
tst.loc[mask_HSA, 'binds'] = preds[mask_HSA][:, 1]
tst.loc[mask_sEH, 'binds'] = preds[mask_sEH][:, 2]



submission = tst[['id', 'binds']].copy()
# 'id'と'binds'列をCSVに出力
submission.to_csv(os.path.join(OUTPUT_DIR,'submission.csv'), index=False)


/tmp/ipykernel_493834/3638645060.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[6.88809081e-08 3.92980269e-07 1.07668195e-04 ... 3.66526801e-05
 8.99846691e-06 5.53948157e-05]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tst.loc[mask_BRD4, 'binds'] = preds[mask_BRD4][:, 0]
